In [6]:
import pandas as pd
import numpy as np

In [7]:
# met office data https://www.metoffice.gov.uk/research/climate/maps-and-data/historic-station-data
df = pd.read_csv("durhamdata.txt", sep="\s+", engine="python", skiprows=6, skipfooter=14)

In [8]:
df['date'] = pd.to_datetime([str(x[0]) + '-' + str(x[1]) + '-01' for x in df.index])
df['year'] = df['date'].dt.year

In [9]:
new_columns = {
    "degC":   "temperature_max_C",
    "degC.1": "temperature_min_C",
    "days":   "days_of_air_frost",
    "mm":     "total_rainfall_mm",
    "hours":  "sunshine_hrs"
}
df.rename(columns=new_columns, inplace=True)
df.head(3)

temperature_max_C temperature_min_C days_of_air_frost  \
1880 1               4.2              -1.4                22   
     2               8.7               0.6                12   
     3               9.2               1.0                12   

       total_rainfall_mm sunshine_hrs       date  year  
1880 1              13.5          --- 1880-01-01  1880  
     2              44.3          --- 1880-02-01  1880  
     3              32.5          --- 1880-03-01  1880

In [10]:
# replace '---' and '*' in the readings
df['temperature_min_C'].replace('---', np.nan, inplace=True)
df['temperature_min_C'] = df['temperature_min_C'].str.replace('*', '', regex=False)
df['temperature_min_C'] = df['temperature_min_C'].astype(float)
df['temperature_max_C'].replace('---', np.nan, inplace=True)
df['temperature_max_C'] = df['temperature_max_C'].str.replace('*', '', regex=False)
df['temperature_max_C'] = df['temperature_max_C'].astype(float)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1704 entries, (1880, 1) to (2021, 12)
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   temperature_max_C  1703 non-null   float64       
 1   temperature_min_C  1703 non-null   float64       
 2   days_of_air_frost  1704 non-null   object        
 3   total_rainfall_mm  1704 non-null   object        
 4   sunshine_hrs       1704 non-null   object        
 5   date               1704 non-null   datetime64[ns]
 6   year               1704 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 103.8+ KB


In [44]:
# create a new column for year
df['year'] = df['date'].dt.year

# group data by year and calculate the average temperature
yearly_df = df.groupby('year').mean().reset_index()

# rename the colums
yearly_df.rename(columns = {"temperature_min_C": "Average yearly minimum temperature",
                            "temperature_max_C": "Average yearly maximum temperature" }, inplace=True)

In [45]:
yearly_df

,year,Average yearly maximum temperature,Average yearly minimum temperature
0,1880,12.333333,3.966667
1,1881,11.500000,3.533333
2,1882,12.500000,4.666667
3,1883,12.672727,4.390909
4,1884,13.100000,4.508333
...,...,...,...
137,2017,13.675000,6.283333
138,2018,13.708333,5.900000
139,2019,13.525000,6.025000
140,2020,13.758333,6.191667


In [46]:
# save csv
yearly_df.to_csv('met_durham.csv', index=False)

In [47]:
import plotly.express as px

# plot the line chart using plotly express
fig = px.scatter(yearly_df, x='year', y=['Average yearly maximum temperature', 'Average yearly minimum temperature'],
                 trendline='ols')

# show the plot
fig.show()